In [596]:
import pandas as pd
import pdf2image
import os
import wget
import glob
import numpy as np
import re
from tika import parser

# Парсинг остальных документов

Для БашГУ:
* ФМиИТ_вещественный, комплексный и функциональный анализ -- сканы
* ФМиИТ_дифференциальные уравнения, динамические системы, оптимальное управление -- магистерская

In [3]:
programs = pd.read_excel('programs.xlsx').fillna('')
inds = programs[programs['ан_тип'] == 'file'].index
parsed = programs.iloc[inds]['ВУЗ'].unique().tolist()
parsed

['АГТУ', 'АГУ', 'БГУ Петровского', 'БашГУ', 'ВУиТ', 'ВолгГТУ']

In [24]:
univers = []
for file in glob.glob('files\*'):
    if file.split('\\')[1] not in parsed:
        univers.append(file.split('\\')[-1])
univers

['АлтГТУ', 'АлтГУ', 'АмГУ', 'БГТУ', 'ВолГУ', 'ИМСИТ']

In [1142]:
# АмГУ -- долг
U = univers[2]
files = []
for file in glob.glob('files\{}\*\*\*.pdf'.format(U)):
    files.append(file)

In [1153]:
file = files[1]
file
len(files)

160

In [1154]:
raw = parser.from_file(file)
text = raw['content'].encode("ansi", "ignore").decode('ansi')
print(text)
















































Аннотация рабочей программы дисциплины «Адаптивный курс математики» 

для направления подготовки 09.03.01 Информатика и вычислительная 

техника 

 

1. Цели и задачи освоения дисциплины (модуля) 
 

Цель дисциплины (модуля): повторение и систематизация математических знаний 

умений и навыков за курс общеобразовательной школы. 

 

Задачи дисциплины (модуля): 

– восполнение пробелов в математическом образовании, которое необходимо для 

успешного овладения математическим материалом в рамках обучения по данному 

направлению бакалавриата;  

– привитие будущим бакалаврам понимания необходимости строгой 

аргументированности математических результатов;  

– выработка навыков грамотной математической речи и правильного применения 

математической символики на материале элементарной математики. 

 

 

2. Компетенции обучающегося, формируемые в результате 
освоения дисциплины и индикаторы их достижения 

Профессиональные компетенции и индика

In [1155]:
text = text.replace('ё', 'е').replace('-\n\n', '').replace('-\t', ' ').replace('\n', ' ').replace('\t', ' ')
len(text)

1978

In [77]:
regex_imsit = [    
    r'.*Цель *?и *?задачи',
    r'Краткая *?характеристика',
    r'Содержание *?дисциплины', 
    r'учебной *?дисциплины',
    r'Место *?дисциплины *?в *?структуре ОПОП.*\(основные блоки, темы\)',
    r'Знания, *?умения *?и *?навыки',
    r'Формы *?проведения.*',
    r'Компетенции',
    r'Задачи *?дисциплины',
    r'изучения *?дисциплины',
    
]

regex_volgu = [    
    r'.*4. Цель и задачи изучения дисциплины',
    r'Цель *?изучения *?дисциплины',
    r'Задачи дисциплины',
    r'Место дисциплины в структуре ОПОП.*\d. Содержание дисциплины',
    r'Выпускник *?должен *?обладать *?следующими *?общепрофессиональными *?компетенциями \(ОПК\)',
    r'Выпускник *?должен *?обладать *?следующими *?универсальными *?компетенциями \(УК\)',
    r'Процесс  изучения  дисциплины  направлен.*ФГОС ВО',
    r'Требования к результатам освоения дисциплины',
    r'Студент должен',
]

regex_altgtu = [
    r'Разработал:.*',
    r'.*В результате освоения дисциплины обучающийся должен обладать следующими компетенциями:',
    r'Контрольный опрос и тест по модулю',
    r'Синтаксис текста',
]

regex_altgu = [
    r'.*Цель изучения дисциплины',
    r'Форма промежуточной аттестации.*',
    r'Место дисциплины в учебном плане .*?  Формируемые',
    r'Используемые информационные, инструментальные и программные средства.*',
    r'Виды учебной работы.*',
    r'Целями изучения дисциплины являются',
    
]

regex_amgu = [
    r'.*Цели и задачи освоения дисциплины \(модуля\)',
    r'Код и наименование профессиональной компетенции',
    r'Код и наименование индикатора дости\-?жения профессиональной компетенции',
    r'Задачи дисциплины',
    r'Целью дисциплины является',
]

regex_bgtu = [    
    r'\(коды, в соответствии с ФГОС ВО\)',
    r'6. ?Автор.*',
    r'.*Форма обучения',
    r'Место дисциплины в структуре ОП?ОП.*Требования к результатам освоения дисциплины',
    r'Общая трудоемкость дисциплины.*\d\d\d? часов',
    r'Общая трудоемкость дисциплины.*\d\d\d? часа?',
    r'Цель дисциплины',
]

regex_general = [
    r'Форма обучения',
    r'Объем дисциплины',
    r'Форма промежуточной аттестации',
    r'Контрольная работа',
    r'Зачет\W', r'Экзамен\W', r'Очная', r'Заочная',
    r'Содержание дисциплины', r'Дисциплина',
    r'стр[.] ?\d\d{0,2}-?\d{0,3}',
    r'cтр[.] ?\d\d{0,2}-?\d{0,3}', # eng c
    r'Тема \d\d?[.]?', r'\d[.]',
    r'\s\d\d?',
    r'Семестр', r'Раздел \w{1,3}\W', 
    r'https?://.*?[.]com/?',
    r'https?://.*?[.]ru/?',
    r'https?://.*?[.]org/?',
    r'www.*?[.]com/?',
    r'www.*?[.]ru/?',
    r'www.*?[.]org/?',
    r'\[.*?\]',
    r'\(\d{1,3}\)',
    r'\(\d{1,3}, \d{1,3}\)',
    r'\s\-', r'\-\s',
    r'[.,:;\"\'«»\(\)*!?\/\\–№]',
]

In [14]:
def brush(text, regex):
    text_ = text
    for reg in regex:
        text_ = re.sub(reg, ' ', text_, flags=re.IGNORECASE)
    for reg in regex_general:
        text_ = re.sub(reg, ' ', text_, flags=re.IGNORECASE)
    
    return text_


def clean_text(text, regex):
    splitted = re.split(r'[ОПКИДУ]{2,3}\-\d\d?', text)
    new_text = brush(splitted[0], regex)
    for i, match in enumerate(re.finditer(r'[ОПКИДУ]{2,3}\-\d\d?', text)):
#         print(match.group(0))
        new_text += ' ' + re.sub(r'[().,:;]', '', match.group(0)) + ' ' + brush(splitted[i + 1], regex)
    return re.sub(r'\ {2,}', ' ', new_text.lstrip().rstrip())

clean_text(text, regex_amgu)

In [1009]:
# import pymorphy2
# morph = pymorphy2.MorphAnalyzer()
# sent = 'дан-ные разра-ботку UX-дизайна'.split()
# for i in range(len(sent)):
#     w = sent[i].replace('-', '')
#     parsed = morph.parse(w)
#     for m in parsed:
#         print(m)
#         break
#     break

In [1158]:
U = univers[2]
regex = regex_amgu

for file in glob.glob('files\{}\*\*\*.pdf'.format(U)):
    raw = parser.from_file(file)
    text = raw['content'].encode("ansi", "ignore").decode('ansi')
    text = text.replace('\n', ' ').replace('\t', ' ').replace('ё', 'е')
    text_ = clean_text(text, regex)
    splitted = file.split('\\')
    new_path = '\\'.join(splitted[:-2]) + '\\' + splitted[-1][:-3].lower() + 'txt'

#     print(new_path)
#     file_ = open(new_path, "w+")
#     file_.write(text_)
#     file_.close()

# Парсинг .docx для БГТУ

In [1197]:
files_bgtu_docx = []
for file in glob.glob('files\БГТУ\*\*\*.docx'):
    if file.split('\\')[-1][:2] == '~$':
        continue
    files_bgtu_docx.append(file)

file = files_bgtu_docx[3]

raw = parser.from_file(file)
text = raw['content'].encode("ansi", "ignore").decode('ansi')
text = text.replace('\n', ' ').replace('\t', ' ').replace('ё', 'е')
text

'                                                 АННОТАЦИЯ рабочей программы дисциплины АРХИТЕКТУРА ВЫЧИСЛИТЕЛЬНЫХ СИСТЕМ Направление подготовки – 09.03.01 «Информатика и вычислительная техника» Профиль – «Программное обеспечение вычислительной техники и автоматизированных систем» Квалификация – бакалавр Форма обучения – очная (для набора 2014 учебного года) 1. Цель дисциплины – познакомить студента с современными методами и подходами к организации компьютерных систем и процессов обработки информации на всех уровнях компьютерных архитектур; сформировать знания об основных классах вычислительных систем и научить выполнять экспресс-анализ их функционирования; познакомить с основополагающими методиками мелкоблочного и крупноблочного распараллеливания сложных задач; познакомить с принципами программирования вычислительных систем с общей памятью в стандарте OpenMP и параллельного программирования в стандарте MPI; научить оценивать эффективность разработанных параллельных алгоритмов. 2. Мес

In [1198]:
regex_bgtu_docx = [    
    r'6. ?Автор.*',
    r'Место дисциплины в структуре ОП?ОП.*Требования к результатам освоения дисциплины',
    r'Общая трудоемкость дисциплины.*\d\d\d? часов',
    r'Общая трудоемкость дисциплины.*\d\d\d? часа?',
    r'.*Цель дисциплины',
]

clean_text(text, regex_bgtu_docx).replace('\xa0', ' ')

'познакомить студента с современными методами и подходами к организации компьютерных систем и процессов обработки информации на всех уровнях компьютерных архитектур сформировать знания об основных классах вычислительных систем и научить выполнять экспресс-анализ их функционирования познакомить с основополагающими методиками мелкоблочного и крупноблочного распараллеливания сложных задач познакомить с принципами программирования вычислительных систем с общей памятью в стандарте OpenMP и параллельного программирования в стандарте MPI научить оценивать эффективность разработанных параллельных алгоритмов Процесс изучения дисциплины направлен на формирование и развитие следующих компетенций способность к самоорганизации и самообразованию ОК-7 способностью инсталлировать программное и аппаратное обеспечение для информационных и автоматизированных систем ОПК-1 способностью осваивать методики использования программных средств для решения практических задач ОПК-2 способностью сопрягать аппаратны

In [1200]:
for file in glob.glob('files\БГТУ\*\*\*.docx'):
    if file.split('\\')[-1][:2] == '~$':
        continue
    raw = parser.from_file(file)
    text = raw['content'].encode("ansi", "ignore").decode('ansi')
    text = text.replace('\n', ' ').replace('\t', ' ').replace('ё', 'е')
    text_ = clean_text(text, regex_bgtu_docx).replace('\xa0', ' ')

    splitted = file.split('\\')
    new_path = '\\'.join(splitted[:-2]) + '\\' + splitted[-1][:-4].lower() + 'txt'

#     print(new_path)
    file_ = open(new_path, "w+")
    file_.write(text_)
    file_.close()

# Правка раннее созданных текстов

In [473]:
parsed

['АГТУ', 'АГУ', 'БГУ Петровского', 'БашГУ', 'ВУиТ', 'ВолгГТУ']

In [753]:
# ВУиТ -- долг -- распарсить pdf
U = 'ВУиТ'
# op = 'ФИиТ_информатика и вычислительная техника'
op = 'ФИиТ_информационные системы и технологии'
files = []
for file in glob.glob('files\{}\{}\*.txt'.format(U, op)):
    files.append(file)
len(files)

52

In [760]:
file = files[10]
print(file, '\n')
try:
    file_ = open(file, 'r')
    text = file_.read()
except:
    print('utf-8')
    file_ = open(file, 'r', encoding='utf-8')
    text = file_.read()
    
file_.close()
text

files\ВУиТ\ФИиТ_информационные системы и технологии\защита информации.txt 



'освоения дисциплины: формирование у студентов знаний в области защиты информации, разграничений доступа к ресурсам; идентификации и аутентификации субъектов; методов и средств криптографической защиты; контроля целостности информации и электронно- цифровой подписи; хранения и распределения ключевой информации; защиты программного обеспечения от несанкционированного использования; защиты от разрушающих программных воздействий; защиты информации в компьютерных сетях; инженерно-технической защиты информации; по руководящим документам России и правовому обеспечению информационной безопасности и противодействию терроризму. Задачи дисциплины: изучение и классификация политик безопасности, свойства безопасности информации, идентификации и аутентификации субъектов, принципов криптографической защиты информации; изучение базовых методов нейтрализации систем защиты от несанкционированного использования безопасности субъектов и объектов, функций хэширования и электронно- цифровой подписи; приобр

In [761]:
regex_vuit = [
    r'[$+]',
    r'.*Целью курса является',
    r'знать,? уметь,? владеть',
    r'Шифр, название компетенции', 
    r'Результаты формирования компетенции',
    r'В результате изучения дисциплины',
    
]

regex_bashgu = [
    r'.*?Целью учебной дисциплины.*?является',
    r'.*?Целью учебной дисциплины.*?является',
    r'.*?Целями изучения дисциплины.*?являются',
    r'.*?Целями освоения дисциплины.*?являются',
    r'.*?Целью освоения дисциплины.*?является',
    r'.*?Целью курса.*?является',
    r'.*?Цель изучения дисциплины',
    r'.*?Целью учебной дисциплины',
    r'Цели изучения дисциплины',
    r'Цель преподавания дисциплины',
    r'Формируемые компетенции',
    r' Освоение дисциплины.*?направлено на формирование следующих компетенций',
    r'В результате освоения дисциплины.*?должны быть сформированы компетенции',
    r'Место дисциплины в структуре.*?Содержание дисциплины \(модуля\)',
    r'Место дисциплины в структуре.*?Содержание дисциплины',
]

regex_bgu_pet = [
    r'.*ЦЕЛИ И ЗАДАЧИ ОСВОЕНИЯ ДИСЦИПЛИНЫ',
    r'Цель освоения дисциплины',
    r'МЕСТО ДИСЦИПЛИНЫ В СТРУКТУРЕ ОПОП.*СОДЕРЖАНИЯ ДИСЦИПЛИНЫ',
    r'СОДЕРЖАНИЕ УЧЕБНОЙ ДИСЦИПЛИНЫ',
    r'ТРЕБОВАНИЯ К РЕЗУЛЬТАТАМ ОСВОЕНИЯ СОДЕРЖАНИЯ ДИСЦИПЛИНЫ',
    r'Процесс изучения дисциплины.*?в соответствии с ФГОС ВО по данному направлению подготовки',
    r'СОДЕРЖАНИЕ УЧЕБНОЙ ДИСЦИПЛИНЫ',
    
]

regex_agu_2 = [
    r'.*направлено на формирование следующих компетенций',
    r'.*обладать следующими компетенциями',
    r'Объем дисциплины.*Основные понятия',
    r'Учебно\-методическое обеспечение.*',
    r'Основная и дополнительная литература.*',
    r'Cамостоятельная работа студентов осуществляется с использованием.*',
    r'лабораторные работы', r'лабораторных работ', r'лабораторное занятие', r'самостоятельной работы',
    r'\d\d?\d? ?часов', r'\d\d?\d? ?часа', r'\d\d?\d? ?час', r'\d\d?\d? ?ч[.]?', r'\d\d? ?з[.] ?е[.]',
    r'лекций|лекция|лекции',
    r'СРС ?\-\d\d?', r'КСР ?\-\d\d?', r'[СП]З ?\-\d\d?',
    r'СРС', r'КСР', r'[СП]З',
    r'практических занятий',
    r'Дисциплина.*в структуре образовательной программы',
    r'Содержание дисциплины',
    r'Контрольное тестирование \d \d\d баллов',
    r'практические занятие', r'практ',
]

regex_agu_1 = [
    r'.*Общекультурные компетенции',
    r'Трудоемкость дисциплины.*Содержание дисциплины',
    r'Объем дисциплины.*Содержание дисциплины',
    r'Общепрофессиональные компетенции', r'Профессиональные компетенции',
    r'.*направлено на формирование следующих компетенций',
    r'Дисциплина относится к.*?в структуре образовательной программы бакалавриата',
    r'Дисциплин.*?относится.*?по ?выбору',
    r'\d\d?\d? ?часов', r'\d\d?\d? ?часа', r'\d\d?\d? ?час', r'\d\d?\d? ?ч[.]', r'\d\d? ?з[.] ?е[.]',
    r'лекций|лекция|лекции',
    r'Планируемые результаты обучения ?по дисциплине',
]

regex_agtu = [
    r'Компетенции обучающегося формируемые в результате освоения дисциплины модуля',
    r'Содержание',
    r'Основные понятия',
    r'Результаты освоения дисциплины модуля',
]


regex_general = [
    r'Форма обучения',
    r'Объем дисциплины',
    r'Форма промежуточной аттестации',
    r'Контрольная работа',
    r'Зачет\W', r'Экзамен\W', r'Очная', r'Заочная',
    r'стр[.] ?\d\d{0,2}-?\d{0,3}',
    r'cтр[.] ?\d\d{0,2}-?\d{0,3}', # eng c
    r'Тема \d\d?[.]?', r'Модуль \d\d?[.]?', r'Модуль [IVXCDL]*', r'\d[.]',
    r'\s\d\d?',
    r'Семестр', r'Раздел \w{1,3}\W', 
    r'https?://.*?[.]com/?',
    r'https?://.*?[.]ru/?',
    r'https?://.*?[.]org/?',
    r'www.*?[.]com/?',
    r'www.*?[.]ru/?',
    r'www.*?[.]org/?',
    r'\[.*?\]',
    r'\(\d{1,3}\)',
    r'\(\d{1,3}, \d{1,3}\)',
    r'\s\-', r'\-\s',
    r'[.,:;\"\'«»\(\)*!?\/\\–—№]',
]

In [762]:
def brush_redef(text, regex=None):
    text_ = re.sub(' {2,}', ' ', text)
    if regex is not None:
        for reg in regex:
            text_ = re.sub(reg, ' ', text_, flags=re.IGNORECASE)
            
    for reg in regex_general:
        text_ = re.sub(reg, ' ', text_, flags=re.IGNORECASE)
    text_ = re.sub(r' {2,}', ' ', text_)
    
    return text_.replace('\u00ad', '')


def clean_text_redef(text, regex=None, fix=False):
    if fix == False:
        splitted = re.split(r'[ОПКИДУ]{2,3}\-\d\d?', text)
        new_text = brush_redef(splitted[0], regex)
        for i, match in enumerate(re.finditer(r'[ОПКИДУ]{2,3}\-\d\d?', text)):
    #         print(match.group(0))
            new_text += ' ' + re.sub(r'[().,:;]', '', match.group(0)) + ' ' + brush_redef(splitted[i + 1], regex)
        return re.sub(r'\ {2,}', ' ', new_text.lstrip().rstrip())
    else:
        splitted = re.split(r'[ОПКИДУ]{2,3} \d\d?', text)
        new_text = brush_redef(splitted[0], regex)
        for i, match in enumerate(re.finditer(r'[ОПКИДУ]{2,3} \d\d?', text)):
            print(match.group(0))
            lit = re.findall('\D{1,}', match.group(0))[0]
            dig = re.findall('\d{1,}', match.group(0))[0]
            fixed = lit[:-1] + '-' + dig
            
            new_text += ' ' + re.sub(r'[().,:;]', '', fixed) + ' ' + brush_redef(splitted[i + 1], regex)
        return re.sub(r'\ {2,}', ' ', new_text.lstrip().rstrip())

    
clean_text_redef(text, regex_vuit, fix=False)

'освоения дисциплины формирование у студентов знаний в области защиты информации разграничений доступа к ресурсам идентификации и аутентификации субъектов методов и средств криптографической защиты контроля целостности информации и электронно цифровой подписи хранения и распределения ключевой информации защиты программного обеспечения от несанкционированного использования защиты от разрушающих программных воздействий защиты информации в компьютерных сетях инженерно-технической защиты информации по руководящим документам России и правовому обеспечению информационной безопасности и противодействию терроризму Задачи дисциплины изучение и классификация политик безопасности свойства безопасности информации идентификации и аутентификации субъектов принципов криптографической защиты информации изучение базовых методов нейтрализации систем защиты от несанкционированного использования безопасности субъектов и объектов функций хэширования и электронно цифровой подписи приобретение практических н

In [763]:
U, op

('ВУиТ', 'ФИиТ_информационные системы и технологии')

In [764]:
regex = regex_vuit

for file in glob.glob('files\{}\{}\*.txt'.format(U, op)):
    flag = 0

    try:
        text = open(file, 'r').read()
    except Exception as e:
        print(e)
        print(file)
        flag = 1
        text = open(file, 'r', encoding='utf-8').read()
        
    text = clean_text_redef(text, regex)
    
    if flag == 1:
        os.remove(file)
        
    file_ = open(file, 'w+', encoding='ansi')
    
    try:
        file_.write(text)
    except Exception as e:
        print('FATAL:', e)
        print(file)
        file_.close() 
        break
    file_.close()    
    
    splitted = file.split('\\')
    file_lower = '\\'.join(splitted[:-1]) + '\\' + splitted[-1].lower()
    os.rename(file, file_lower)

In [516]:
# i = 0

In [517]:
# from IPython.display import display, clear_output

# alls = glob.glob('files\{}\*\*.txt'.format(U))
# for i in range(len(alls)):
#     clear_output(wait=False)
    
#     file = open(alls[i], 'r')
#     print(alls[i], '\n')
#     print(file.read())
#     file.close()
#     _ = input()
#     if _ == 'break':
#         print(i)
#         break